In [ ]:
!git clone https://github.com/KuzmaKhrabrov/character-tokenizer.git

Cloning into 'character-tokenizer'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 20 (delta 5), reused 10 (delta 3), pack-reused 0
Receiving objects: 100% (20/20), 5.89 KiB | 1.96 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [ ]:
!pip install transformers -q

In [ ]:
import sys
sys.path.append("/content/character-tokenizer/")
# sys.path.append("/kaggle/working/character-tokenizer")

In [ ]:
import torch
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import os
from charactertokenizer import CharacterTokenizer

Задание: обучите модель классификации букв для задачи расстановки ударения с помощью методов из библиотеки transformers. Датасет для обучения можно взять отсюда: https://github.com/Koziev/NLP_Datasets/blob/master/Stress/all_accents.zip

1. Напишите класс для Dataset/Dataloder и разбейте данные на случайные train / test сплиты в соотношении 50:50. (1 балл)
2. Попробуйте обучить одну или несколько из моделей: Bert, Albert, Deberta. Посчитайте метрику Accuracy на train и test. (1 балл). При преодолении порога в Accuracy на test 0.8: (+1 балл), 0.85: (+2 балла), 0.89: (+3 балла).
Пример конфигурации для deberta: https://huggingface.co/IlyaGusev/ru-word-stress-transformer/blob/main/config.json

In [ ]:
if not os.path.exists('all_accents.tsv'):
  url = "https://github.com/Koziev/NLP_Datasets/raw/master/Stress/all_accents.zip"
  !wget $url -q
  !unzip all_accents.zip
  !rm all_accents.zip

Archive:  all_accents.zip
  inflating: all_accents.tsv         


In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
chars = "АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя"
model_max_length = 64
tokenizer = CharacterTokenizer(chars, model_max_length)

In [ ]:
class StressDataset(torch.utils.data.Dataset):
    def __init__(self, path, tokenizer):
        self.df = pd.read_csv(path, delimiter='\t')
        self.tokenizer = tokenizer
        self.max_length = tokenizer.model_max_length

    def __getitem__(self, idx):
        word, stress = self.df.iloc[idx]
        encoded_dict = self.tokenizer.encode_plus(
                    word,                          # Sentence to encode.
                    add_special_tokens = True,     # Add '[CLS]' and '[SEP]'
                    max_length = self.max_length, # Pad & truncate all sentences.
                    padding='max_length',
                    return_attention_mask = True,  # Construct attn. masks.
                    return_tensors = 'pt',         # Return pytorch tensors.
                   )
        label = self.get_label(stress)
        input_ids = encoded_dict['input_ids'].squeeze(0)
        attention_mask = encoded_dict['attention_mask'].squeeze(0)
        return input_ids, attention_mask, label

    def __len__(self):
        return len(self.df)

    def get_label(self, stress):
        stress_count = 0
        label = [-100] * self.max_length
        word_len = len(stress) - stress.count('^')
        label[1:word_len+1] = [0] * word_len

        for i, c in enumerate(stress):
            if c == '^':
                label[i + 1 - stress_count] = 1
                stress_count += 1
        return torch.tensor(label)

In [ ]:
dataset = StressDataset('all_accents.tsv', tokenizer)

In [ ]:
from torch.utils.data import random_split


train_size = int(0.5 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

840,267 training samples
840,267 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it
# here. For fine-tuning BERT on a specific task, the authors recommend a batch
# size of 16 or 32.
batch_size = 32

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order.
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size, # Trains with this batch size.
            drop_last=True
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size, # Evaluate with this batch size.
        )

In [ ]:
from transformers import DebertaV2ForTokenClassification, DebertaConfig, AdamW


config_dict = {
    "architectures": ["DebertaV2ForTokenClassification"],
    "attention_probs_dropout_prob": 0.1,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 1024,
    "id2label": {"0": "NO", "1": "STRESS"},
    "initializer_range": 0.02,
    "intermediate_size": 4096,
    "label2id": {"NO": 0, "STRESS": 1},
    "layer_norm_eps": 1e-07,
    "max_length": 64,
    "max_position_embeddings": 66,
    "max_relative_positions": -1,
    "model_type": "deberta-v2",
    "num_attention_heads": 8,
    "num_hidden_layers": 4,
    "pad_token_id": 4,
    "pooler_dropout": 0,
    "pooler_hidden_act": "gelu",
    "pooler_hidden_size": 1024,
    "pos_att_type": None,
    "position_biased_input": True,
    "relative_attention": False,
    "type_vocab_size": 0,
    "vocab_size": tokenizer.vocab_size
}

config = DebertaConfig.from_dict(config_dict)

# Создание модели на основе конфигурации
model = DebertaV2ForTokenClassification(config)
model.cuda()

DebertaV2ForTokenClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(73, 1024, padding_idx=4)
      (position_embeddings): Embedding(66, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-3): 4 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 70 different named parameters.

==== Embedding Layer ====

deberta.embeddings.word_embeddings.weight                 (73, 1024)
deberta.embeddings.position_embeddings.weight             (66, 1024)
deberta.embeddings.LayerNorm.weight                          (1024,)
deberta.embeddings.LayerNorm.bias                            (1024,)
deberta.encoder.layer.0.attention.self.query_proj.weight (1024, 1024)

==== First Transformer ====

deberta.encoder.layer.0.attention.self.query_proj.bias       (1024,)
deberta.encoder.layer.0.attention.self.key_proj.weight  (1024, 1024)
deberta.encoder.layer.0.attention.self.key_proj.bias         (1024,)
deberta.encoder.layer.0.attention.self.value_proj.weight (1024, 1024)
deberta.encoder.layer.0.attention.self.value_proj.bias       (1024,)
deberta.encoder.layer.0.attention.output.dense.weight   (1024, 1024)
deberta.encoder.layer.0.attention.output.dense.bias          (1024,)
deberta.encoder.layer.0.attention.output.LayerNorm.weight     

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 5e-5, eps = 1e-8)

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4.
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def accuracy(preds, labels):
    preds = np.argmax(preds, axis=2)
    true_preds = [
        [p for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(preds, labels)
    ]
    true_labels= [
        [l for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(preds, labels)
    ]
    s = 0
    for pred, label in zip(true_preds, true_labels):
        if pred == label:
            s += 1
    return s / len(labels)

In [ ]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss,
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 1000 batches.
        if step % 100 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            print(f'  Average Loss: {total_train_loss / step}')

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because
        # accumulating the gradients is "convenient while training RNNs".
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here:
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        output = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value
        # from the tensor.
        total_train_loss += output.loss.item()

        # Perform a backward pass to calculate the gradients.
        output.loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        # Unpack this training batch from our dataloader.
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids
        #   [1]: attention masks
        #   [2]: labels
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            output = model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)

        # Accumulate the validation loss.
        total_eval_loss += output.loss.item()

        # Move logits and labels to CPU
        logits = output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += accuracy(logits, label_ids)


    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 2 ========
Training...
  Batch   100  of  26,258.    Elapsed: 0:00:14.
  Average Loss: 0.2224211063981056
  Batch   200  of  26,258.    Elapsed: 0:00:26.
  Average Loss: 0.19335158694535493
  Batch   300  of  26,258.    Elapsed: 0:00:37.
  Average Loss: 0.1790248303115368
  Batch   400  of  26,258.    Elapsed: 0:00:49.
  Average Loss: 0.17005230886861683
  Batch   500  of  26,258.    Elapsed: 0:01:01.
  Average Loss: 0.16388145512342453
  Batch   600  of  26,258.    Elapsed: 0:01:12.
  Average Loss: 0.1589123268549641
  Batch   700  of  26,258.    Elapsed: 0:01:24.
  Average Loss: 0.15454400038080557
  Batch   800  of  26,258.    Elapsed: 0:01:36.
  Average Loss: 0.15087683737277985
  Batch   900  of  26,258.    Elapsed: 0:01:47.
  Average Loss: 0.1478652508225706
  Batch 1,000  of  26,258.    Elapsed: 0:01:59.
  Average Loss: 0.1453603562042117
  Batch 1,100  of  26,258.    Elapsed: 0:02:11.
  Average Loss: 0.1432896817949685
  Batch 1,200  of  26,258.    Elapsed: 

#### Итоговая точность на валидации:

In [ ]:
training_stats[-1]['Valid. Accur.']

0.9155790525499483

#### Точность на трейне:

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
model.eval()
total_train_accuracy = 0

for batch in tqdm(train_dataloader):
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    with torch.no_grad():
        output = model(b_input_ids,
                       token_type_ids=None,
                       attention_mask=b_input_mask,
                       labels=b_labels)

    logits = output.logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    total_train_accuracy += accuracy(logits, label_ids)

avg_train_accuracy = total_train_accuracy / len(train_dataloader)
print(f"Train accuracy: {avg_train_accuracy}")

  0%|          | 0/26258 [00:00<?, ?it/s]

Train accuracy: 0.9252882454870897


In [ ]:
# state = {
#     'state_dict': model.state_dict(),
# }
# torch.save(state, 'deberta_91_accuracy.pth')

In [ ]:
def test(word):
    encoded_dict = tokenizer.encode_plus(
            word,
            add_special_tokens = True,
            max_length = 64,
            padding='max_length',
            return_attention_mask = True,
            return_tensors = 'pt',
           )
    input_ids = encoded_dict['input_ids'].to(device)
    attention_mask = encoded_dict['attention_mask'].to(device)
    output = model(input_ids,
                token_type_ids=None,
                attention_mask=attention_mask)
    label = output.logits.detach().cpu().numpy().argmax(axis=2)[0]
    input_ids = input_ids.cpu().numpy()[0]

    letters = list(word)
    stresses_count = 0
    for i, stress in enumerate(label[1:1+len(word)]):
        if stress:
            letters.insert(i + stresses_count, '^')
            stresses_count += 1
    return ''.join(letters)

In [ ]:
def test_sentence(sentence):
    words = sentence.split()
    return ' '.join([test(word) for word in words])

In [ ]:
test_sentence('Трансформер обучен!')

'Трансф^ормер об^учен!'

In [ ]:
test_sentence('Домашнее задание выполнено.')

'Дом^ашнее зад^ание в^ыполнено.'

In [ ]:
test_sentence('Хочу на стажировочку во ВКонтакте')

'Хоч^у н^а стажир^овочку в^о ВКонт^акте'